<a href="https://colab.research.google.com/github/AnoVando/MSIS/blob/master/MSIS521_IA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt') # tokenizer
nltk.download('wordnet') # lemmatizer
nltk.download('stopwords') # used to handle words like a, an, the
nltk.download('averaged_perceptron_tagger') # Part of Speech

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
from sklearn.cluster import AgglomerativeClustering



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
url = 'https://github.com/AnoVando/MSIS/raw/master/IA3.csv'
data = pd.read_csv(url, header='infer')
reviews = data['review'].tolist()

In [3]:
print(data[:5])

   id                                             review       label
0   1  About the shop: There is a restaurant in Soi L...  restaurant
1   2  About the shop: Through this store for about t...  restaurant
2   3  Roast Coffee &amp; Eatery is a restaurant loca...  restaurant
3   4  Eat from the children. The shop is opposite. P...  restaurant
4   5  The Ak 1 shop at another branch tastes the sam...  restaurant


Part 1. Topic Model
There are 1000 reviews for restaurants and films in a collection under the attached csv file. All of those
reviews are saved as text files. In this assignment, you are required to investigate the topics of those
reviews. In particular, please follow the steps listed below:
1. Transform those reviews into a term‐document matrix, lemmatize all the words, remove the
stop‐words and punctuations, set the minimal document frequency for each term to be 5 and
include 2‐gram.
2. Use the LDA model to extract the topics of each document. In particular, we assume there are 6
topics.
3. Report the topic distribution and the top‐2 topics of the first 10 restaurant reviews (id = [1:10])
and the first 10 movie reviews (id = [501:510]).
4. Find the top‐5 terms (terms with the top‐5 highest weights) for each of the 6 topics. Based on
those terms, describe what those topics are about.
5. Based on finding in 3 and 4, describe what review 1 [ID=1] and review 501 [ID=501] are about?
Please submit 1 file:
A word file includes python code with your comment #, and one screenshot on your Jupyter
Notebook showing that your code has run through successfully for each of the first four steps (4
screenshots in total). Also, report your answers to question 3, 4, and 5 at the end of the word
file.

1. Transform those reviews into a term‐document matrix, lemmatize all the words, remove the
stop‐words and punctuations, set the minimal document frequency for each term to be 5 and
include 2‐gram.

In [0]:
# Tokenize, Lemmatize and Remove Stop Words
lemmatizer = nltk.stem.WordNetLemmatizer()
processed = []
for review in reviews:
    tokens = nltk.word_tokenize(review.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if not token in stopwords.words('english')]
    processed.append(tokens)


In [5]:
# Generate TF-IDF Vectors
processed_tfidf = [" ".join(x) for x in processed]

tfidf = TfidfVectorizer(ngram_range=(2, 2), min_df=5) # 2-grams and min. document frequency of 5
tfidf.fit(processed_tfidf)
vector = tfidf.transform(processed_tfidf)
vector = vector.toarray()
print(tfidf.vocabulary_)
print(vector)
print(vector.shape)

{'restaurant decorated': 1499, 'average price': 74, 'price baht': 1254, 'soup sweet': 1646, 'sweet taste': 1726, 'cream cheese': 231, 'eat together': 342, 'topic lt': 1840, 'gt atmosphere': 641, 'atmosphere nice': 67, 'lt food': 934, 'food taste': 531, 'taste gt': 1741, 'gt lt': 643, 'lt service': 935, 'service gt': 1585, 'gt good': 642, 'good service': 629, 'service good': 1584, 'good food': 605, 'lt value': 936, 'price quite': 1259, 'quite expensive': 1275, 'atmosphere good': 66, 'good delicious': 601, 'delicious food': 269, 'like please': 853, 'please forgive': 1236, 'forgive think': 537, 'think one': 1780, 'one comment': 1103, 'comment win': 212, 'win competition': 2011, 'competition good': 219, 'good restaurant': 627, 'restaurant thank': 1508, 'three year': 1798, 'first time': 512, 'time try': 1825, 'restaurant located': 1505, 'located soi': 882, 'bottle baht': 141, 'chicken wing': 180, 'spicy taste': 1659, 'united state': 1883, 'good enough': 603, 'value money': 1886, 'money gt':

2. Use the LDA model to extract the topics of each document. In particular, we assume there are 6
topics.

In [6]:
dictionary = gensim.corpora.Dictionary(processed)
bow_corpora = [dictionary.doc2bow(doc) for doc in processed]
print(bow_corpora[1])

lda_model = gensim.models.LdaModel(bow_corpora, num_topics=6, id2word=dictionary)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

lda_model[bow_corpora[1]]

[(1, 1), (2, 2), (4, 1), (8, 1), (9, 1), (18, 1), (21, 3), (22, 1), (26, 3), (27, 4), (28, 1), (33, 1), (34, 2), (35, 4), (37, 1), (40, 1), (41, 2), (49, 1), (51, 1), (52, 2), (53, 1), (57, 3), (58, 1), (59, 1), (63, 1), (67, 2), (68, 1), (70, 2), (73, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (84, 2), (85, 2), (88, 1), (89, 1), (90, 2), (95, 1), (98, 1), (99, 1), (100, 3), (101, 2), (102, 2), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 2), (110, 1), (111, 1), (112, 1), (113, 1), (114, 1), (115, 1), (116, 1), (117, 1), (118, 1), (119, 1), (120, 1), (121, 1), (122, 1), (123, 2), (124, 1), (125, 1), (126, 2), (127, 1), (128, 1), (129, 1), (130, 1), (131, 1), (132, 1), (133, 1), (134, 1), (135, 1), (136, 1), (137, 1), (138, 1), (139, 1), (140, 1), (141, 1), (142, 1), (143, 1), (144, 1), (145, 1), (146, 1), (147, 1), (148, 1), (149, 2), (150, 1), (151, 1), (152, 1), (153, 3), (154, 1), (155, 1)]
Topic: 0 
Words: 0.026*"quot" + 0.012*"film" + 0.008*"people" + 0.

[(0, 0.14950535), (1, 0.8453608)]

3. Report the topic distribution and the top‐2 topics of the first 10 restaurant reviews (id = [1:10])
and the first 10 movie reviews (id = [501:510]).

In [42]:
url = 'https://github.com/AnoVando/MSIS/raw/master/IA3.csv'
data = pd.read_csv(url, header='infer')
data1 = data[0:9]
data2 = data[500:509]
data = data1.append(data2)
reviews = data['review'].tolist()

# Tokenize, Lemmatize and Remove Stop Words
lemmatizer = nltk.stem.WordNetLemmatizer()
processed = []
for review in reviews:
    tokens = nltk.word_tokenize(review.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if not token in stopwords.words('english')]
    processed.append(tokens)

text = gensim.corpora.Dictionary(processed)
corpus = [text.doc2bow(doc) for doc in processed]

lda_model = gensim.models.LdaModel(corpus, num_topics=2, id2word=text)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.046*"quot" + 0.009*"film" + 0.007*"wa" + 0.007*"like" + 0.006*"people" + 0.006*"lone" + 0.005*"dolphin" + 0.005*"ha" + 0.005*"good" + 0.005*"ranger"
Topic: 1 
Words: 0.018*"quot" + 0.011*"film" + 0.008*"good" + 0.007*"like" + 0.006*"price" + 0.005*"taste" + 0.005*"eat" + 0.005*"delicious" + 0.005*"dolphin" + 0.005*"baht"


In [0]:
# install pyLDAvis if necessary
# !pip install pyLDAvis

In [0]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpora, dictionary)
vis